In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#pd.set_option("display.notebook_repr_html", False)
#pd.set_option("display.max_rows",100)
#pd.set_option("expand_frame_repr", True)
#pd.set_option('max_colwidth',80)


weather = pd.read_csv(
  "http://www.gagolewski.com/resources/data/nycflights13_weather.csv.gz",  
  compression="gzip",
  comment="#"
)
flights = pd.read_csv(
  "http://www.gagolewski.com/resources/data/nycflights13_flights.csv.gz",  
  compression="gzip",
  comment="#"
)
airlines = pd.read_csv(
  "http://www.gagolewski.com/resources/data/nycflights13_airlines.csv.gz",  
  compression="gzip",
  comment="#"
)
planes = pd.read_csv(
  "http://www.gagolewski.com/resources/data/nycflights13_planes.csv.gz",  
  compression="gzip",
  comment="#"
)
airports = pd.read_csv(
  "http://www.gagolewski.com/resources/data/nycflights13_airports.csv.gz",  
  compression="gzip",
  comment="#"
)

In [2]:
dfs = {'weather'   : weather,
       'flights'   :  flights,
       'airlines'  : airlines,
       'planes'    :  planes,
       'airports'  :   airports,
      }

In [10]:
for k in dfs:
    print("\n DATA FRAME")
    print(k)
    print(len(dfs[k]))
    #print(dfs[k].head())


 DATA FRAME
weather
26130

 DATA FRAME
flights
336776

 DATA FRAME
airlines
16

 DATA FRAME
planes
3322

 DATA FRAME
airports
1458


In [11]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(r'pandas_multiple.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
with writer as writer:
    for k in dfs:
        df = dfs[k]
        #print("\n DATA FRAME")
        print(k)
        #print(dfs[k].head())

        df.to_excel(writer, sheet_name=k)

# Close the Pandas Excel writer and output the Excel file.
#writer.save()

weather
flights
airlines
planes
airports


In [13]:
import sqlite3
import os, os.path
os.getcwd()

'C:\\Users\\jon\\git\\ref_DataScienceRetreat\\DSR Lecture notebooks\\00_ Advanced_Python'

In [14]:
conn = sqlite3.connect("nycflights13.db")

In [16]:
q = """
CREATE TABLE airlines(
    carrier CHAR(2) PRIMARY KEY,
    name VARCHAR(64)
    )
"""
conn.execute(q)
conn.commit()

In [17]:
a = list(airlines.to_records(index=False))
a[:3]

[('9E', 'Endeavor Air Inc.'),
 ('AA', 'American Airlines Inc.'),
 ('AS', 'Alaska Airlines Inc.')]

In [19]:
q = ("""
    INSERT INTO airlines(carrier,name) VALUES (?, ?)
""", a)
conn.executemany(*q)
conn.commit()

In [20]:
q = """
SELECT * FROM airlines WHERE name LIKE '%AMERICA%'
"""
cur = conn.execute(q)
w = cur.fetchall()
w

[('AA', 'American Airlines Inc.'), ('VX', 'Virgin America')]

In [21]:
pd.DataFrame.from_records(w)

,0,1
0,AA,American Airlines Inc.
1,VX,Virgin America


In [25]:
conn.close()

In [26]:
if os.path.isfile("nycflights13.db"):
    os.remove("nycflights13.db")
conn = sqlite3.connect("nycflights13.db")

In [27]:
conn = sqlite3.connect("nycflights13.db")
weather.to_sql("weather",conn)
flights.to_sql("flights",conn)
airlines.to_sql("airlines",conn)
planes .to_sql("planes",conn)
airports.to_sql("airports",conn)

In [28]:
pd.read_sql_query("""
    PRAGMA TABLE_INFO(airlines)
""",conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,carrier,TEXT,0,None,0
2,2,name,TEXT,0,None,0


#### 1) 

In [29]:
pd.read_sql_query("""
    SELECT DISTINCT engine FROM PLANES
""",conn)

,engine
0,Turbo-fan
1,Turbo-jet
2,Reciprocating
3,4 Cycle
4,Turbo-shaft
5,Turbo-prop


In [84]:
pd.DataFrame(planes["engine"].drop_duplicates().reset_index(drop=True))

,engine
0,Turbo-fan
1,Turbo-jet
2,Reciprocating
3,4 Cycle
4,Turbo-shaft
5,Turbo-prop


In [90]:
planes.loc[:,['engine']].drop_duplicates().reset_index(drop=True)

,engine
0,Turbo-fan
1,Turbo-jet
2,Reciprocating
3,4 Cycle
4,Turbo-shaft
5,Turbo-prop


In [87]:
pd.DataFrame(planes["engine"].unique(), columns = ['engine'])

,engine
0,Turbo-fan
1,Turbo-jet
2,Reciprocating
3,4 Cycle
4,Turbo-shaft
5,Turbo-prop


#### 2) 

In [30]:
pd.read_sql_query("""
    SELECT DISTINCT type, engine FROM PLANES
""",conn)

,type,engine
0,Fixed wing multi engine,Turbo-fan
1,Fixed wing multi engine,Turbo-jet
2,Fixed wing single engine,Reciprocating
3,Fixed wing multi engine,Reciprocating
4,Fixed wing single engine,4 Cycle
5,Rotorcraft,Turbo-shaft
6,Fixed wing multi engine,Turbo-prop


In [43]:
pd.DataFrame(planes[['type',"engine"]].drop_duplicates())

,type,engine
0,Fixed wing multi engine,Turbo-fan
51,Fixed wing multi engine,Turbo-jet
424,Fixed wing single engine,Reciprocating
427,Fixed wing multi engine,Reciprocating
686,Fixed wing single engine,4 Cycle
811,Rotorcraft,Turbo-shaft
1045,Fixed wing multi engine,Turbo-prop


#### 3) 

In [32]:
pd.read_sql_query("""
    SELECT COUNT(*) engine FROM planes GROUP BY engine
""",conn)

,engine
0,2
1,28
2,2750
3,535
4,2
5,5


In [95]:
planes.groupby('engine').size().reset_index().rename(columns={0:"COUNT"})

,engine,COUNT
0,4 Cycle,2
1,Reciprocating,28
2,Turbo-fan,2750
3,Turbo-jet,535
4,Turbo-prop,2
5,Turbo-shaft,5


In [74]:
pd.DataFrame(planes['engine'].groupby(planes['engine']).count())

,engine
engine,
4 Cycle,2
Reciprocating,28
Turbo-fan,2750
Turbo-jet,535
Turbo-prop,2
Turbo-shaft,5


#### 4) 

In [97]:
pd.crosstab(planes.engine, planes.type)

type,Fixed wing multi engine,Fixed wing single engine,Rotorcraft
engine,,,
4 Cycle,0,2,0
Reciprocating,5,23,0
Turbo-fan,2750,0,0
Turbo-jet,535,0,0
Turbo-prop,2,0,0
Turbo-shaft,0,0,5


In [98]:
pd.crosstab(planes.engine, planes.type).unstack()

type                      engine       
Fixed wing multi engine   4 Cycle             0
                          Reciprocating       5
                          Turbo-fan        2750
                          Turbo-jet         535
                          Turbo-prop          2
                          Turbo-shaft         0
Fixed wing single engine  4 Cycle             2
                          Reciprocating      23
                          Turbo-fan           0
                          Turbo-jet           0
                          Turbo-prop          0
                          Turbo-shaft         0
Rotorcraft                4 Cycle             0
                          Reciprocating       0
                          Turbo-fan           0
                          Turbo-jet           0
                          Turbo-prop          0
                          Turbo-shaft         5
dtype: int64

In [50]:
pd.read_sql_query("""
    SELECT COUNT(*) engine  FROM PLANES GROUP BY engine,type
""",conn)

,engine
0,2
1,5
2,23
3,2750
4,535
5,2
6,5


#### 5) 

In [75]:
pd.read_sql_query("""
    SELECT COUNT(*), engine, type FROM planes GROUP BY engine, type
""",conn)

,COUNT(*),engine,type
0,2,4 Cycle,Fixed wing single engine
1,5,Reciprocating,Fixed wing multi engine
2,23,Reciprocating,Fixed wing single engine
3,2750,Turbo-fan,Fixed wing multi engine
4,535,Turbo-jet,Fixed wing multi engine
5,2,Turbo-prop,Fixed wing multi engine
6,5,Turbo-shaft,Rotorcraft


In [96]:
planes.groupby(['engine','type']).size().reset_index().rename(columns={0:"COUNT"})

,engine,type,COUNT
0,4 Cycle,Fixed wing single engine,2
1,Reciprocating,Fixed wing multi engine,5
2,Reciprocating,Fixed wing single engine,23
3,Turbo-fan,Fixed wing multi engine,2750
4,Turbo-jet,Fixed wing multi engine,535
5,Turbo-prop,Fixed wing multi engine,2
6,Turbo-shaft,Rotorcraft,5


In [82]:
planes.head()
planes.describe()

,year,engines,seats,speed,c
count,3252.000000,3322.000000,3322.000000,23.000000,3322.0
mean,2000.484010,1.995184,154.316376,236.782609,1.0
std,7.193425,0.117593,73.654974,149.759794,0.0
min,1956.000000,1.000000,2.000000,90.000000,1.0
25%,1997.000000,2.000000,140.000000,107.500000,1.0
50%,2001.000000,2.000000,149.000000,162.000000,1.0
75%,2005.000000,2.000000,182.000000,432.000000,1.0
max,2013.000000,4.000000,450.000000,432.000000,1.0


In [83]:
planes['c'] = 1
planes[['engine','type','c','seats']].groupby(['engine','type']).count()

c  seats
engine        type                                 
4 Cycle       Fixed wing single engine     2      2
Reciprocating Fixed wing multi engine      5      5
              Fixed wing single engine    23     23
Turbo-fan     Fixed wing multi engine   2750   2750
Turbo-jet     Fixed wing multi engine    535    535
Turbo-prop    Fixed wing multi engine      2      2
Turbo-shaft   Rotorcraft                   5      5